# Algorithmically Assign Abx To CSNs based on Model Predictions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import os, glob

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/conorcorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

### Load Predictions Data For Each Abx Option

Predictions of coverage for each antibiotic selection are stored in csv files using the following directory schema.  This function reads in predictions for each classifier so that each row is CSN and has an estimated probability of coverage for each antibiotic selection.  We implement a test function that compute the AUROC of each model after the predictions have been read in and cross checks it with the AUROC that was computed and stored in an `auroc.txt` text file during the model training procedure

In [ ]:
import pdb
from sklearn.metrics import roc_auc_score

def test_load_predictions(df):
    """
    Reads in output of df, computes AUROC for each classifier and asserts that it equals the AUROC
    listed in the auroc.txt file associate with the classfier's directory
    """
    base_path="/Users/conorcorbin/repos/er_infection/results/ast_models_bucket1/testing/{abx}"
    abx_options = ["Vancomycin",
               "Ampicillin",
               "Cefazolin",
               "Ceftriaxone",
               "Cefepime",
               "Zosyn",
               "Ciprofloxacin",
               "Meropenem",
               "Vancomycin_Meropenem",
               "Vancomycin_Zosyn",
               "Vancomycin_Cefepime",
               "Vancomycin_Ceftriaxone"
               ]
    for abx in abx_options:
        path = base_path.format(abx=abx)
        f_auroc = os.path.join(path, 'auroc.txt')
        with open(f_auroc, 'r') as f:
            auroc = round(float(f.read()), 3)
        
        computed_auroc = round(roc_auc_score(df['%s_label' % abx], df['%s_predictions' % abx]), 3)
        
        assert auroc == computed_auroc
        print("%s_auroc: %s"% (abx, str(auroc)))

def load_predictions():
    """Helper function that loads predictions from AST classifiers for test set data"""
    
    base_path="/Users/conorcorbin/repos/er_infection/results/ast_models_bucket1/testing/{abx}"
    abx_options = ["Vancomycin",
                   "Ampicillin",
                   "Cefazolin",
                   "Ceftriaxone",
                   "Cefepime",
                   "Zosyn",
                   "Ciprofloxacin",
                   "Meropenem",
                   "Vancomycin_Meropenem",
                   "Vancomycin_Zosyn",
                   "Vancomycin_Cefepime",
                   "Vancomycin_Ceftriaxone"
                   ]
    df = pd.DataFrame()
    for i, abx in enumerate(abx_options):
        path = base_path.format(abx=abx)
        f_path = glob.glob(os.path.join(path, '*predictions.csv'))[0]
        if i == 0:
            df = pd.read_csv(f_path)
            df = df[['anon_id', 'pat_enc_csn_id_coded', 'label', 'predictions']]
            df = df.rename(columns={'label' : '%s_label' % abx,
                                    'predictions' : '%s_predictions' % abx})
        else:
            df_preds = pd.read_csv(f_path)
            df_preds = df_preds[['anon_id', 'pat_enc_csn_id_coded', 'label', 'predictions']]
            df_preds = df_preds.rename(columns={'label' : '%s_label' % abx,
                                                'predictions' : '%s_predictions' % abx})
            df = df.merge(df_preds, how='left', on=['anon_id', 'pat_enc_csn_id_coded'])
    
    return df
    
df = load_predictions()
test_load_predictions(df)

In [ ]:
print(len(df))
df.head()

### Get clinician prescribing patterns
This SQL query gathers all abx medications ordered within the first 24 hours of admission that were administered to the patient in long format ( one row per administered med_description ) and then joins to our labels table so that we can cross check whether the administered antibiotic was sufficient to cover the patient. 

In [ ]:
query = """
SELECT
    om.anon_id, om.pat_enc_csn_id_coded, om.order_med_id_coded, l.index_time, om.med_description,
    l.Ampicillin, l.Ciprofloxacin, l.Cefazolin, l.Ceftriaxone, l.Cefepime, l.Zosyn, l.Vancomycin,
    l.Meropenem, l.Vancomycin_Meropenem, l.Vancomycin_Zosyn, l.Vancomycin_Cefepime, l.Vancomycin_Ceftriaxone
FROM
    `mining-clinical-decisions.abx.abx_orders_given_and_stopped` om
INNER JOIN 
    `mining-clinical-decisions.abx.final_ast_labels` l
USING
    (pat_enc_csn_id_coded)
WHERE
    om.was_given = 1
ORDER BY 
    om.anon_id, om.pat_enc_csn_id_coded, om.order_time
"""
query_job = client.query(query)
df_abx = query_job.result().to_dataframe()
df_abx.head()

### Aggregate antibiotic orders 
Here we aggregate the antibiotic orders so that one row in the desulting dataframe corresponds to a unique CSN. We do this by
1. Grouping by the CSN
2. Grabbing the first word (antibiotic name) from the med description
3. Aggregating the `med_description` column such that it is a single string with all antibiotics admistered to the patient, sorted in alphabetical order and separated by spaces. 
4. Only keep CSNs where the set of administered antibiotics is equal to one of the antbiotic selections we've trained classifiers for. 

#### List Most Commmon Antibiotic Combinations

In [ ]:
concat_abx = lambda x : ' '.join(np.unique(sorted([a for a in x])))
df_common_abx = (df_abx
    .assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description]) # Only Take first word (abx)
    .assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                        .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                       for a in x.med_description])
    .assign(year=lambda x: x.index_time.dt.year) # get year of each CSN - used to filter later on
    .groupby('pat_enc_csn_id_coded')
    .agg({'med_description' : concat_abx})
    .reset_index()
    .groupby('med_description')
    .agg(num_csns=('pat_enc_csn_id_coded', 'nunique'))
    .sort_values('num_csns', ascending=False)
           )
df_common_abx.head(20)

In [ ]:
# Useful dictionaries to map corresponding. strings for the sameantibiotic selections
abx_map = {'Ceftriaxone' : "CEFTRIAXONE",
           'Vancomycin_Zosyn' : "PIPERACILLIN-TAZOBACTAM VANCOMYCIN",
           'Zosyn' : "PIPERACILLIN-TAZOBACTAM",
           'Vancomycin_Ceftriaxone' : "CEFTRIAXONE VANCOMYCIN",
           'Vancomycin_Cefepime' : "CEFEPIME VANCOMYCIN",
           'Cefepime' : "CEFEPIME",
           'Vancomycin' :  "VANCOMYCIN",
           'Meropenem' : "MEROPENEM",
           'Vancomycin_Meropenem' : "MEROPENEM VANCOMYCIN",
           'Cefazolin' : "CEFAZOLIN",
           'Ciprofloxacin' : "CIPROFLOXACIN",
           'Ampicillin' : 'AMPICILLIN'
          }
abx_map_inverse = {abx_map[key] : key for key in abx_map}
abx_map_inverse['CEFTRIAXONE PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Zosyn'
# abx_map_inverse['LEVOFLOXACIN PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Zosyn'
abx_map_inverse['AZITHROMYCIN PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Zosyn'
# abx_map_inverse['MEROPENEM PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Meropenem'
abx_map_inverse['AZITHROMYCIN CEFTRIAXONE'] = 'Ceftriaxone'
years = [2019]
# Lambda that aggregate Antibiotic orders after we've grouped by CSN
concat_abx = lambda x : ' '.join(np.unique(sorted([a for a in x])))

# 
df_drugs = (df_abx
    .assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description]) # Only Take first word (abx)
    .assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                        .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                       for a in x.med_description])
    .assign(year=lambda x: x.index_time.dt.year) # get year of each CSN - used to filter later on
    .groupby('pat_enc_csn_id_coded')
    .agg({'med_description' : concat_abx,
          'year' : 'first',
          'Ampicillin' : 'first',
          'Ciprofloxacin' : 'first',
          'Cefazolin' : 'first',
          'Ceftriaxone' : 'first',
          'Cefepime' : 'first',
          'Zosyn' : 'first',
          'Vancomycin' : 'first',
          'Meropenem' : 'first',
          'Vancomycin_Ceftriaxone' : 'first',
          'Vancomycin_Cefepime' : 'first',
          'Vancomycin_Zosyn' : 'first',
          'Vancomycin_Meropenem' : 'first'})
    .reset_index()
    # Only look at test set data and CSNs where allowed antibiotic selection was administered
    .query("year == @years and med_description in @abx_map_inverse", engine='python') 
    .assign(med_description=lambda x: [abx_map_inverse[a] for a in x.med_description])
)

# Roughly 700 of the 1300 original CSNs in the test set
print(len(df_drugs))
df_drugs.head()

### Merge this dataframe to predictions dataframe
After this step we should have a dataframe that has one row per CSN, each row should have the antibiotic selection actually administered to the patient, along with the predicted probability of said antibiotic selection covering the patient, and the ground truth as to whether it did. 

In [ ]:
# Merge df_total to df on pat_enc_csn_id_coded
df_new = (df
    .merge(df_drugs, how='inner', on='pat_enc_csn_id_coded')
)

# Sanity check - make sure %abx_label columns are equal to %abx columns
for abx in abx_map:
    for i in range(len(df_new)):
        assert df_new[abx].values[i] == df_new['%s_label' % abx].values[i]
        
# Sanity check 2: compute AUROC of this subset of patients and compare to AUROC on full test set
base_path = "/Users/conorcorbin/repos/er_infection/results/ast_models_bucket1/testing/{abx}"
for abx in abx_map:
    computed_auroc = roc_auc_score(df_new['%s_label' % abx], df_new['%s_predictions' % abx])
    f_auroc = os.path.join(base_path.format(abx=abx), 'auroc.txt')
    with open(f_auroc, 'r') as f:
        auroc = float(f.read())
    print("{}: Full test set AUROC:{:.3f} Subset AUROC:{:.3f}".format(abx, auroc, computed_auroc))

### Create Binary Integer Programming Problem Formulation and Solve
Here we specificy the problem formulation of the optimization process we wish to solve. The goal is to maximize the probability of covering the set of patients in the test set with the available antibiotic selections subject to the constraints that we assign each antibiotic selection a prespecified number of times, and that we only assign one antibiotic selection to each patient CSN. 

More technically, Let $N$ be the number of patient CSNs in our test set who were administered one of the 12 abx selections by clinicians, and let $K$ be the number of possible antibiotic selections.  Let $A$ be a matrix in $\mathbb{R}^{N\times K}$ such that $a_{ij}$ is 1 if antibiotic selection $j$ is selected for patient CSN $i$ and 0 otherwise. Let $\Phi$ be a matrix in $\mathbb{R}^{N \times K}$ such that $\phi_{ij}$ is the predicted probability that antibiotic $j$ will cover patient CSN $i$.  Let $C$ be a vector in $\mathbb{R}^K$ such that $c_j$ specifies the budget for anitbiotic selection $j$ - that is the number of times we are allowed to select antibiotic $j$ across our $N$ patient CSNs. Our problem formulation is as follows. 

$$  \underset{A}{\text{maximize}} \sum_{i=1}^{N} \sum_{j=1}^K \phi_{ij} a_{ij} $$

Subject to the following constraints:

$$ \sum_{j=1}^{K} a_{ij} = 1 \quad i = 1, ..., N $$

$$ \sum_{i=1}^{N} a_{ij} = c_j \quad j = 1, ...,  K $$

In the following code, we implenent and solve this optimization process using the pulp python package. 

#### How often was each options prescribed by clinicians?

In [ ]:
(df_new
.groupby('med_description')
.agg(num_csns=('pat_enc_csn_id_coded', 'nunique'))
)

In [ ]:
abx_options = ["Vancomycin",
           "Ampicillin",
           "Cefazolin",
           "Ceftriaxone",
           "Cefepime",
           "Zosyn",
           "Ciprofloxacin",
           "Meropenem",
           "Vancomycin_Meropenem",
           "Vancomycin_Zosyn",
           "Vancomycin_Cefepime",
           "Vancomycin_Ceftriaxone"
           ]

abx_model = LpProblem("Antibiotics", LpMaximize)

# Create binary indicators for whether treatment is used
drug_inds = {}
for abx in abx_options:
    drug_inds[abx] = [LpVariable('%s_%d' % (abx, i), lowBound=0, upBound=1, cat='Binary')
                      for i in range(len(df_new))]

# Add objective function to model
per_csn_sum = []
for i in range(len(df_new)):
    _sum = 0
    for abx in abx_options:
        _sum += drug_inds[abx][i] * df_new['%s_predictions' % abx].values[i]
    per_csn_sum.append(_sum)
    
abx_model += lpSum(per_csn_sum)

# Add one selection constraint
for i in range(len(df_new)):
    selections = []
    for abx in abx_options:
        selections.append(drug_inds[abx][i])
    abx_model += lpSum(selections) == 1

# Add max assignment constraints
# abx_assignment_constraints = {"Vancomycin" : 13,
#                               "Ampicillin" : 0,
#                               "Cefazolin" : 8,
#                               "Ceftriaxone" : 367,
#                               "Cefepime" : 14,
#                               "Zosyn" : 102,
#                               "Ciprofloxacin" : 8,
#                               "Meropenem" : 9,
#                               "Vancomycin_Meropenem" : 9,
#                               "Vancomycin_Zosyn" :  113,
#                               "Vancomycin_Cefepime" : 23,
#                               "Vancomycin_Ceftriaxone" : 31
#                              }
abx_assignment_constraints = {"Vancomycin" : 13,
                              "Ampicillin" : 0,
                              "Cefazolin" : 8,
                              "Ceftriaxone" : 404,
                              "Cefepime" : 14,
                              "Zosyn" : 102,
                              "Ciprofloxacin" : 8,
                              "Meropenem" : 9,
                              "Vancomycin_Meropenem" : 9,
                              "Vancomycin_Zosyn" :  149,
                              "Vancomycin_Cefepime" : 23,
                              "Vancomycin_Ceftriaxone" : 31
                             }


for drug in drug_inds:
    abx_model += lpSum([drug_inds[drug][i] for i in range(len(df_new))]) == abx_assignment_constraints[drug]

# Solve model
abx_model.solve()
print("Status:", LpStatus[abx_model.status])

# Save selected antibiotic to df_new
abx_decisions = []
for i in range(len(df_new)):
    abx_decision = None
    for abx in abx_options:
        if drug_inds[abx][i].value() == 1:
            abx_decision = abx
    assert abx_decision is not None
    abx_decisions.append(abx_decision)
df_new['IP_med_description'] = abx_decisions


### Compare Performance to Clinician Performance
1. Write a function that takes in antibiotic selection and outputs a 1 if that selection covered the patient.  Simple to do, but annoying because of different ways we've named antibiotic selections.
2. Compute fraction of time each patient CSN was covered by the antibiotic selection. 

In [ ]:
# Ugly helper function that just does some string mapping
def compute_was_covered(x, decision_column='med_description'):
    """
    Given med description, find appropriate label column and return whether patient was covered during CSN
    Returns "Not in abx options" if abx regimen isn't in our set of 12 options - useful for filtering later
    """
    if decision_column == 'med_description':
        med_description = x.med_description
    elif decision_column == 'random_med_description':
        med_description = x.random_med_description
    elif decision_column == 'IP_med_description':
        med_description = x.IP_med_description
    
    return x[med_description]
#     if med_description == "CEFTRIAXONE":
#         return x.Ceftriaxone
#     elif med_description == "PIPERACILLIN-TAZOBACTAM VANCOMYCIN":
#         return x.Vancomycin_Zosyn
#     elif med_description == "PIPERACILLIN-TAZOBACTAM":
#         return x.Zosyn
#     elif med_description == "CEFTRIAXONE VANCOMYCIN":
#         return x.Vancomycin_Ceftriaxone
#     elif med_description == "CEFEPIME VANCOMYCIN":
#         return x.Vancomycin_Cefepime
#     elif med_description == "CEFEPIME":
#         return x.Cefepime
#     elif med_description == "VANCOMYCIN":
#         return x.Vancomycin
#     elif med_description == "MEROPENEM":
#         return x.Meropenem
#     elif med_description == "MEROPENEM VANCOMYCIN":
#         return x.Vancomycin_Meropenem
#     elif med_description == "CEFAZOLIN":
#         return x.Cefazolin
#     elif med_description == "CIPROFLOXACIN":
#         return x.Ciprofloxacin
#     elif med_description == "AMPICILLIN":
#         return x.Ampicillin
#     else:
#         return "Not in abx options"
    
    
# Create flag for whether clinicians covered the patient during the csn, whether a random assignemnt covered patient
# CSN, and whether optimized assignment covered the patient CSN.

df_new = (df_new
    .assign(random_med_description=lambda x: np.random.choice(x.med_description, size=len(x.med_description), replace=False))
)
df_new = (df_new
    #.sample(frac=1.0, replace=True) # bootstrap each iteration
    .assign(was_covered_dr=df_new.apply(lambda x: compute_was_covered(x), axis=1))
    .assign(was_covered_random=df_new.apply(lambda x: compute_was_covered(x, 
                                                                          decision_column='random_med_description'),
                                                                          axis=1))
    .assign(was_covered_IP=df_new.apply(lambda x: compute_was_covered(x, 
                                                                      decision_column='IP_med_description'),
                                                                      axis=1))
)

clin_covered_rate = df_new['was_covered_dr'].sum() / len(df_new)
random_covered_rate = df_new['was_covered_random'].sum() / len(df_new)
ip_covered_rate = df_new['was_covered_IP'].sum() / len(df_new)

print(clin_covered_rate)
print(random_covered_rate)
print(ip_covered_rate)

df_new_random = (df_new
        .groupby('random_med_description')
        .agg(num_distinct_csns=('pat_enc_csn_id_coded', 'count'),
             num_times_covered_random=('was_covered_random', 'sum'))
        .reset_index()
        .assign(random_covered=lambda x: ['{}/{}'.format(c, t) for c, t in zip(x.num_times_covered_random,
                                                                               x.num_distinct_csns)])
        .rename(columns={'random_med_description' : 'med_description'})
)[['med_description', 'random_covered']]
                 
df_new_clinician = (df_new
        .groupby('med_description')
        .agg(num_distinct_csns=('pat_enc_csn_id_coded', 'count'),
             num_times_covered_dr=('was_covered_dr', 'sum'))
        .reset_index()
        .assign(dr_covered=lambda x: ['{}/{}'.format(c, t) for c, t in zip(x.num_times_covered_dr,
                                                                               x.num_distinct_csns)])
)[['med_description', 'dr_covered']]
                    
df_new_ip = (df_new
        .groupby('IP_med_description')
        .agg(num_distinct_csns=('pat_enc_csn_id_coded', 'count'),
             num_times_covered_IP=('was_covered_IP', 'sum'))
        .reset_index()
        .assign(IP_covered=lambda x: ['{}/{}'.format(c, t) for c, t in zip(x.num_times_covered_IP,
                                                                               x.num_distinct_csns)])
        .rename(columns={'IP_med_description' : 'med_description'})
)[['med_description', 'IP_covered']]

df_new_agg = (df_new_random
    .merge(df_new_clinician, how='inner', on='med_description')
    .merge(df_new_ip, how='inner', on='med_description')
)

df_new_agg